<a href="https://colab.research.google.com/github/PLanet-09AI/isaziIndaba/blob/main/SHERPARD.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install roboflow

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 80.3/80.3 kB 1.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.8/66.8 kB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.5/54.5 kB 3.3 MB/s eta 0:00:00
  Attempting uninstall: idna
    Found existing installation: idna 3.10
    Uninstalling idna-3.10:
      Successfully uninstalled idna-3.10


In [2]:
pip install roboflow

In [3]:
pip install roboflow

In [4]:
!pip install roboflow


In [5]:
from roboflow import Roboflow


In [6]:
!pip install supervision

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 151.5/151.5 kB 1.5 MB/s eta 0:00:00


In [7]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [29]:
import os
import cv2
import numpy as np
from sklearn.preprocessing import LabelEncoder
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Input
from tensorflow.keras.optimizers import Adam

In [30]:
training_folder = '/content/Training'
testing_folder = '/content/Testing'


In [31]:
def load_images_from_folder(folder):
    images = []
    labels = []
    for filename in os.listdir(folder):
        img = cv2.imread(os.path.join(folder, filename))
        if img is not None:
            images.append(img)
            # Assuming labels are part of the filename, e.g., '0_bottle.jpg' or '1_bottle.jpg'
            label = filename.split('_')[0]
            labels.append(label)
    return images, labels

X_train, y_train = load_images_from_folder(training_folder)
X_test, y_test = load_images_from_folder(testing_folder)


In [32]:
def preprocess_images(images):
    processed_images = []
    for img in images:
        # Resize image to 224x224 (common size for models like VGG, ResNet)
        img_resized = cv2.resize(img, (224, 224))
        # Normalize pixel values to [0, 1]
        img_normalized = img_resized / 255.0
        processed_images.append(img_normalized)
    return np.array(processed_images)

X_train = preprocess_images(X_train)
X_test = preprocess_images(X_test)

In [33]:
# Convert all labels to strings
y_train = [str(label) for label in y_train]
y_test = [str(label) for label in y_test]


In [34]:
# Combine labels from both training and testing datasets
all_labels = y_train + y_test

In [35]:
# Fit the LabelEncoder on the combined labels
le = LabelEncoder()
le.fit(all_labels)

LabelEncoder()

In [36]:
# Transform the labels
y_train_encoded = le.transform(y_train)
y_test_encoded = le.transform(y_test)


In [37]:
# Define your model
model = Sequential([
    Input(shape=(224, 224, 3)),  # Use Input layer to specify input shape
    Conv2D(32, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Flatten(),
    Dense(64, activation='relu'),
    Dense(1, activation='sigmoid')
])

# Compile the model with the correct learning rate parameter
model.compile(optimizer=Adam(learning_rate=0.001), loss='binary_crossentropy', metrics=['accuracy'])

# Train the model
model.fit(X_train, y_train_encoded, epochs=10, batch_size=32, validation_data=(X_test, y_test_encoded))

Epoch 1/10
15/15 ━━━━━━━━━━━━━━━━━━━━ 43s 3s/step - accuracy: 0.7697 - loss: -76.4742 - val_accuracy: 0.5625 - val_loss: -280.8302
Epoch 2/10
15/15 ━━━━━━━━━━━━━━━━━━━━ 80s 3s/step - accuracy: 0.7318 - loss: -482.8569 - val_accuracy: 0.5625 - val_loss: -1005.9858
Epoch 3/10
15/15 ━━━━━━━━━━━━━━━━━━━━ 42s 3s/step - accuracy: 0.7185 - loss: -2920.6526 - val_accuracy: 0.5625 - val_loss: -3957.0801
Epoch 4/10
15/15 ━━━━━━━━━━━━━━━━━━━━ 82s 3s/step - accuracy: 0.7088 - loss: -9272.4629 - val_accuracy: 0.5625 - val_loss: -10060.3477
Epoch 5/10
15/15 ━━━━━━━━━━━━━━━━━━━━ 39s 3s/step - accuracy: 0.7104 - loss: -21183.6133 - val_accuracy: 0.5625 - val_loss: -29929.3242
Epoch 6/10
15/15 ━━━━━━━━━━━━━━━━━━━━ 44s 3s/step - accuracy: 0.6822 - loss: -92841.4141 - val_accuracy: 0.5625 - val_loss: -63719.0781
Epoch 7/10
15/15 ━━━━━━━━━━━━━━━━━━━━ 81s 3s/step - accuracy: 0.7301 - loss: -118193.4453 - val_accuracy: 0.5625 - val_loss: -107368.3828
Epoch 8/10
15/15 ━━━━━━━━━━━━━━━━━━━━ 40s 3s/step - accur

In [44]:
from sklearn.metrics import classification_report

y_pred = (model.predict(X_test) > 0.5).astype("int32")
print(classification_report(y_test_encoded, y_pred))


1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step
              precision    recall  f1-score   support

           0       0.00      0.00      0.00         6
           1       0.56      1.00      0.72         9
           9       0.00      0.00      0.00         1

    accuracy                           0.56        16
   macro avg       0.19      0.33      0.24        16
weighted avg       0.32      0.56      0.40        16



Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.


In [45]:
from sklearn.utils.class_weight import compute_class_weight

class_weights = compute_class_weight('balanced', classes=np.unique(y_train_encoded), y=y_train_encoded)
class_weight_dict = dict(enumerate(class_weights))

In [39]:
model.save('/content/Trained_Model/my_model.h5')


In [41]:
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense
from tensorflow.keras.models import Model
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import Adam

In [43]:
print(f"Training samples: {train_generator.samples}")
print(f"Testing samples: {test_generator.samples}")


Training samples: 0
Testing samples: 0


In [46]:
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam

# Load MobileNetV2 with pre-trained weights
base_model = MobileNetV2(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
base_model.trainable = False

# Add custom layers on top
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(1024, activation='relu')(x)
predictions = Dense(1, activation='sigmoid')(x)

# Define the model
model = Model(inputs=base_model.input, outputs=predictions)

# Compile the model
model.compile(optimizer=Adam(learning_rate=0.0001), loss='binary_crossentropy', metrics=['accuracy'])

# Train the model with class weights
model.fit(train_generator, epochs=10, validation_data=test_generator, class_weight=class_weight_dict)


ValueError: Must provide at least one structure